## 1. Gather the Data
Answer:
The dataset has been successfully loaded from the provided URL. The data contains information about food delivery orders in New Delhi, including columns like order_id, delivery_method, commission_fee, order_value, payment_method, delivery_time, and refunds_chargebacks.

In [18]:
import pandas as pd
import numpy as np
from scipy import stats

# Load the dataset
url = "https://statso.io/wp-content/uploads/2024/02/food_orders_new_delhi.csv"
df = pd.read_csv(url)

## 2. Clean the Dataset
**Answer:**
The dataset was cleaned by:

Removing rows with missing values.

Standardizing categorical values (e.g., delivery_method and payment_method to lowercase).

Ensuring numeric columns (commission_fee, order_value, delivery_time) are correctly typed.

In [24]:
import pandas as pd

# Load the dataset
df = pd.read_csv("food_orders_new_delhi.csv")

# Display basic info
print("Initial Dataset Info:")
print(df.info())

# Step 1: Handle missing values
missing_values = df.isnull().sum()
print("\nMissing Values Before Handling:\n", missing_values)

# Drop rows where essential columns (Order ID, Customer ID, Restaurant ID) are missing
df.dropna(subset=['Order ID', 'Customer ID', 'Restaurant ID'], inplace=True)

# Fill missing numerical values with median
num_cols = ['Order Value', 'Delivery Fee', 'Commission Fee', 'Payment Processing Fee', 'Refunds/Chargebacks']
df[num_cols] = df[num_cols].fillna(df[num_cols].median())

# Fill missing categorical values with mode
cat_cols = ['Discounts and Offers']
df[cat_cols] = df[cat_cols].apply(lambda x: x.fillna(x.mode()[0]))

# Step 2: Remove duplicate records
df.drop_duplicates(inplace=True)

# Step 3: Convert data types
df['Order Date and Time'] = pd.to_datetime(df['Order Date and Time'])
df['Delivery Date and Time'] = pd.to_datetime(df['Delivery Date and Time'])

# Step 4: Identify and handle outliers using IQR method
for col in num_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]

# Display cleaned dataset info
print("\nCleaned Dataset Info:")
print(df.info())

# Save the cleaned dataset
df.to_csv("food_orders_cleaned.csv", index=False)

print("\nData Cleaning Completed. Cleaned dataset saved as 'food_orders_cleaned.csv'.")

Initial Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Order ID                1000 non-null   int64 
 1   Customer ID             1000 non-null   object
 2   Restaurant ID           1000 non-null   object
 3   Order Date and Time     1000 non-null   object
 4   Delivery Date and Time  1000 non-null   object
 5   Order Value             1000 non-null   int64 
 6   Delivery Fee            1000 non-null   int64 
 7   Payment Method          1000 non-null   object
 8   Discounts and Offers    815 non-null    object
 9   Commission Fee          1000 non-null   int64 
 10  Payment Processing Fee  1000 non-null   int64 
 11  Refunds/Chargebacks     1000 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 93.9+ KB
None

Missing Values Before Handling:
 Order ID                    0
Customer ID             

## 3. Next Step: Most Common Delivery Method
Now, let's find:

The most common delivery method.
The proportion of the most common method.

In [27]:
# Find the most common payment method
most_common_method = df['Payment Method'].mode()[0]
method_count = df['Payment Method'].value_counts()[most_common_method]
total_orders = len(df)

# Calculate the proportion
proportion = method_count / total_orders

print(f"Most common payment method: {most_common_method}")
print(f"Proportion of most common payment method: {proportion:.2%}")

Most common payment method: Cash on Delivery
Proportion of most common payment method: 35.64%


# 4. Next Step: **95% Confidence Interval for the Proportion**
calculate the **95% confidence interval** for the proportion of orders made with the most common payment method.
the formula for the confidence interval for a proportion:
The 95% confidence interval for the proportion is given by:

\[
CI = p \pm Z \times \sqrt{\frac{p(1 - p)}{n}}
\]

Where:
- \( p \) is the sample proportion
- \( Z \) is the Z-score for a 95% confidence level (**1.96**)
- \( n \) is the total number of orders

In [43]:
import numpy as np
from scipy.stats import norm

# Given values
p = proportion  # 35.64% as a decimal
n = total_orders
z = norm.ppf(0.975)  # Z-score for 95% confidence

# Standard error calculation
se = np.sqrt((p * (1 - p)) / n)

# Confidence interval
lower_bound = p - z * se
upper_bound = p + z * se

print(f"95% Confidence Interval for proportion: ({lower_bound:.4f}, {upper_bound:.4f})")
print(f"Or as percentages: ({lower_bound*100:.2f}%, {upper_bound*100:.2f}%)")

95% Confidence Interval for proportion: (0.3253, 0.3874)
Or as percentages: (32.53%, 38.74%)


# 5. Next Step: Average & Median Commission Fee
 let's calculate:  
1. **Average commission fee** (Mean)  
2. **Median commission fee**  

In [77]:
# Calculate average commission fee
average_commission_fee = df['Commission Fee'].mean()

# Calculate median commission fee
median_commission_fee = df['Commission Fee'].median()

print(f"Average Commission Fee: {average_commission_fee:.2f}")
print(f"Median Commission Fee: {median_commission_fee:.2f}")

Average Commission Fee: 126.76
Median Commission Fee: 127.00


# 6. Average Order Value of the Customer
To find the average order value for each customer, we can group by the Customer ID and calculate the mean of Order Value:

In [80]:
# Calculate average order value per customer
average_order_value_per_customer = df.groupby('Customer ID')['Order Value'].mean()

# Calculate the overall average order value
overall_average_order_value = average_order_value_per_customer.mean()

print(f"Average Order Value per Customer: {overall_average_order_value:.2f}")

Average Order Value per Customer: 1045.00


# 7.Next Step: Probability Calculations
Find the probability that a commission fee is:

More than 120
Less than 143
Between 86 and 133

In [83]:
# Import necessary library
from scipy.stats import norm

# Get mean and standard deviation
mean_commission = df['Commission Fee'].mean()
std_commission = df['Commission Fee'].std()

# Probability calculations using normal distribution
prob_more_than_120 = 1 - norm.cdf(120, mean_commission, std_commission)
prob_less_than_143 = norm.cdf(143, mean_commission, std_commission)
prob_between_86_and_133 = norm.cdf(133, mean_commission, std_commission) - norm.cdf(86, mean_commission, std_commission)

print(f"Probability that commission fee is more than 120: {prob_more_than_120:.4f}")
print(f"Probability that commission fee is less than 143: {prob_less_than_143:.4f}")
print(f"Probability that commission fee is between 86 and 133: {prob_between_86_and_133:.4f}")

Probability that commission fee is more than 120: 0.5624
Probability that commission fee is less than 143: 0.6472
Probability that commission fee is between 86 and 133: 0.3860


# 8. Next Step: Average Delivery Time for Credit Card Orders
Calculate the average delivery time for orders that were paid using Credit Card.

first create a new column that computes the delivery time (difference between delivery time and order time), then filter by Credit Card payment method and find the average.

In [86]:
# Calculate delivery time (difference between delivery and order times)
df['Delivery Time'] = (df['Delivery Date and Time'] - df['Order Date and Time']).dt.total_seconds() / 60  # in minutes

# Filter for Credit Card payment method
credit_card_orders = df[df['Payment Method'] == 'Credit Card']

# Calculate average delivery time for Credit Card orders
average_delivery_time_cc = credit_card_orders['Delivery Time'].mean()

print(f"Average Delivery Time for Credit Card orders: {average_delivery_time_cc:.2f} minutes")

Average Delivery Time for Credit Card orders: 73.87 minutes


# 9. Next Step: Lowest Order Value of 10% Largest Orders (Cash on Delivery)
find the lowest order value of the 10% largest orders paid using Cash on Delivery.:

Filter for Cash on Delivery orders.
Sort them by order value in descending order.
Select the top 10% and find the minimum order value.

In [89]:
# Filter for Cash on Delivery payment method
cash_on_delivery_orders = df[df['Payment Method'] == 'Cash on Delivery']

# Sort by Order Value in descending order
sorted_cash_on_delivery = cash_on_delivery_orders.sort_values(by='Order Value', ascending=False)

# Get the lowest order value of the top 10% largest orders
top_10_percent = sorted_cash_on_delivery.head(int(len(sorted_cash_on_delivery) * 0.1))
lowest_order_value_top_10 = top_10_percent['Order Value'].min()

print(f"Lowest order value of the 10% largest Cash on Delivery orders: {lowest_order_value_top_10}")


Lowest order value of the 10% largest Cash on Delivery orders: 1810


# 10. Next Step: Highest Order Value of 60% Least Orders (Digital Wallet)
calculate the highest order value of the 60% least orders paid using Digital Wallet.
My plan:

Filter for Digital Wallet payment method.
Sort the orders by order value in ascending order.
Select the bottom 60% and find the maximum order value.

In [95]:
# Filter for Digital Wallet payment method
digital_wallet_orders = df[df['Payment Method'] == 'Digital Wallet']

# Sort by Order Value in ascending order
sorted_digital_wallet = digital_wallet_orders.sort_values(by='Order Value', ascending=True)

# Get the highest order value of the bottom 60% least orders
bottom_60_percent = sorted_digital_wallet.head(int(len(sorted_digital_wallet) * 0.6))
highest_order_value_bottom_60 = bottom_60_percent['Order Value'].max()

print(f"Highest order value of the 60% least Digital Wallet orders: {highest_order_value_bottom_60}")

Highest order value of the 60% least Digital Wallet orders: 1166


# 11. Mean order value between Digital Wallet and Cash on Delivery.

In [98]:
# Calculate mean order value for Digital Wallet
mean_order_value_dw = digital_wallet_orders['Order Value'].mean()

# Calculate mean order value for Cash on Delivery
mean_order_value_cod = cash_on_delivery_orders['Order Value'].mean()

# Find the difference
mean_difference = mean_order_value_dw - mean_order_value_cod

print(f"Mean order value for Digital Wallet: {mean_order_value_dw:.2f}")
print(f"Mean order value for Cash on Delivery: {mean_order_value_cod:.2f}")
print(f"Difference in mean order value (Digital Wallet - Cash on Delivery): {mean_difference:.2f}")

Mean order value for Digital Wallet: 1030.64
Mean order value for Cash on Delivery: 1039.66
Difference in mean order value (Digital Wallet - Cash on Delivery): -9.01


# 12. Next Step: Delivery Time Claim Test
Let’s test the claim: "The delivery time for any item will take more than 53 minutes" using a significance level of 0.05.

perform a one-sample t-test to check if the mean delivery time is significantly greater than 53 minutes.

In [101]:
from scipy import stats

# Perform a one-sample t-test to check if mean delivery time > 53 minutes
# H0: mean delivery time <= 53
# H1: mean delivery time > 53

# Calculate the mean delivery time
mean_delivery_time = df['Delivery Time'].mean()

# Perform t-test (one-sample)
t_statistic, p_value = stats.ttest_1samp(df['Delivery Time'], 53)

# Since it's a one-sided test, we check if the p-value is less than 0.05
if p_value / 2 < 0.05 and mean_delivery_time > 53:
    result = "Reject the null hypothesis: The delivery time is significantly greater than 53 minutes."
else:
    result = "Fail to reject the null hypothesis: The delivery time is not significantly greater than 53 minutes."

print(f"Mean delivery time: {mean_delivery_time:.2f} minutes")
print(f"t-statistic: {t_statistic:.2f}")
print(f"p-value: {p_value:.4f}")
print(result)

Mean delivery time: 73.60 minutes
t-statistic: 23.90
p-value: 0.0000
Reject the null hypothesis: The delivery time is significantly greater than 53 minutes.


Since the p-value is less than 0.05, we reject the null hypothesis. This means there is strong evidence to support the claim that the delivery time is significantly greater than 53 minutes.

## 13. Next Step: Range of Middle 50% of Ordering-Delivery Durations
calculate the range within which the middle 50% of the ordering-delivery durations (i.e., the interquartile range) differ.

In [105]:
# Calculate the interquartile range (IQR) of delivery times
iqr_delivery_time = df['Delivery Time'].quantile(0.75) - df['Delivery Time'].quantile(0.25)

print(f"Range within which the middle 50% of ordering-delivery durations differ: {iqr_delivery_time:.2f} minutes")

Range within which the middle 50% of ordering-delivery durations differ: 46.00 minutes


# 14. Next Step: Distribution of Refunds/Chargebacks
Analyze the distribution of payment methods where the Refunds/Chargebacks are greater than zero.

filter for rows where the Refunds/Chargebacks column has values greater than zero and then look at the distribution of Payment Method.

In [108]:
# Filter for rows where Refunds/Chargebacks > 0
refunds_data = df[df['Refunds/Chargebacks'] > 0]

# Get the distribution of Payment Methods for these rows
payment_method_distribution = refunds_data['Payment Method'].value_counts()

print("Distribution of payment methods for Refunds/Chargebacks > 0:")
print(payment_method_distribution)

Distribution of payment methods for Refunds/Chargebacks > 0:
Payment Method
Cash on Delivery    69
Credit Card         65
Digital Wallet      63
Name: count, dtype: int64
